# Model Training

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk, Sequence, Value
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EvalPrediction,
    TrainerCallback
)
import evaluate
import torch
from torch.nn import BCEWithLogitsLoss
from typing import Optional, Dict
from torch import FloatTensor, tensor
import logging
from sklearn.metrics import classification_report, hamming_loss
from transformers import EvalPrediction

## Dataset Preparation

Will be training 3 datasets and comparing performances:
1. goEmotions + other datasets + textattack data augmentation
2. goEmotions + other datasets
3. original goEmotions dataset

In [2]:
random_seed = 1234

In [3]:
labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

label2id = {label:idx for idx, label in enumerate(labels)}
id2label = {idx:label for idx, label in enumerate(labels)}

### Load base model and tokenizer

In [4]:
model_name = "distilbert/distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=len(labels),     
    problem_type="multi_label_classification", # uses BCEWithLogitsLoss by default)
    id2label=id2label,
    label2id=label2id
) 

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# padding will be dynamically done in batching with DataCollatorWithPadding
def tokenize_func(examples):
    return tokenizer(examples['text'], truncation=True)

### Load datasets
Will be training 3 different datasets
- goEmotions (go)
- goEmotions + other datasets (merged)
- goEmotions + other datasets + textattack data augmentation (augmented)

In [6]:
# augmented
dataset = load_from_disk('./datasets/cleaned_hf/augmented_hf')
# # merged
# dataset = load_from_disk('./datasets/cleaned_hf/merged_hf')
# # go
# dataset = load_from_disk('./datasets/cleaned_hf/goEmotions_cleaned')

In [7]:
type(dataset['train']['labels'][1][0])

int

In [8]:
# In order to use BCEWithLogitsLoss, we need to convert labels to float or it'll give errors
# https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
dataset = dataset.cast_column('labels', Sequence(feature=Value(dtype='float32')))

In [9]:
type(dataset['train']['labels'][1][6])

float

In [10]:
# before shuffle
dataset['train']['text'][:5]

["I'm sorry to hear, come see me when you die.",
 'i feel terrified because even if i have the time to write out how i feel about mr',
 'I asked at the Bodies Reveled show if they used prisoner bodies.. I got an awkward no.',
 'i feel awful for making this all about me and my flawed academia instilled value system but my brain won t shut up about it',
 'i feel so repressed when compared to dear a href http eurodancemix']

In [11]:
# shuffle training set
dataset['train'] = dataset['train'].shuffle(seed=random_seed)

In [12]:
# after shuffle
dataset['train']['text'][:5]

["I gave up trying to be 'normal' years ago. But I get your point though.",
 '@blue_north27 http://twitpic.com/4jcjr - Mmm yummy... looks like an invitation to me',
 'I feel a connection to this woman',
 'My friends are awesome! @JNBlack @koreantomcruise -- and the non Twitter ones here right now too!!',
 'People like you is also why no players want to play and stay in Orlando']

#### Calculate Class Weights

In [13]:
from generate_class_weights import generate_class_weights 

In [14]:
class_weights = generate_class_weights(dataset['train']['labels'], multi_class=False, one_hot_encoded=True)
class_weights

{0: 0.7691456243514355,
 1: 1.3645066273932254,
 2: 0.49946091644204854,
 3: 1.2860613071139386,
 4: 1.0808341029504691,
 5: 1.779591836734694,
 6: 1.401840877569033,
 7: 1.4498272152311404,
 8: 2.988308023115173,
 9: 1.549547038327526,
 10: 1.5710046629927936,
 11: 0.9356616873553545,
 12: 6.0391091797935905,
 13: 2.286948472693613,
 14: 0.8414758751182593,
 15: 1.1933025652033917,
 16: 25.210884353741495,
 17: 0.3419344917730278,
 18: 0.4463357353620105,
 19: 12.078218359587181,
 20: 0.8910438789821679,
 21: 19.488168273444348,
 22: 1.7916364515349288,
 23: 1.1817602040816326,
 24: 3.397402597402597,
 25: 0.26198527245949926,
 26: 0.8437108708025043,
 27: 0.15054127427948574}

#### Tokenization

In [15]:
# tokenize in batch
tokenized_dataset = dataset.map(tokenize_func, batched=True)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 88944
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 10426
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 12721
    })
})

#### Batching and Dynamic padding

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Training
- Will be using `Trainer` instead of `SFTTrainer` because `SFTTrainer` is often for llms.

In [17]:
training_args = TrainingArguments(
    './models/',
    eval_strategy="epoch",
    )

#### Compute metrics function

In [18]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
# https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    
    # Generate classification report
    report = classification_report(
        y_true=labels,
        y_pred=y_pred,
        output_dict=True,  # Convert report to a dictionary
        zero_division=0  # Avoid division errors for missing labels
    )

    # Extract key metrics
    micro_precision = report["micro avg"]["precision"]
    micro_recall = report["micro avg"]["recall"]
    micro_f1 = report["micro avg"]["f1-score"]

    macro_precision = report["macro avg"]["precision"]
    macro_recall = report["macro avg"]["recall"]
    macro_f1 = report["macro avg"]["f1-score"]

    weighted_precision = report["weighted avg"]["precision"]
    weighted_recall = report["weighted avg"]["recall"]
    weighted_f1 = report["weighted avg"]["f1-score"]

    hamming = hamming_loss(y_true, y_pred)

    # Combine all metrics into a dictionary
    metrics = {
        "micro_precision": micro_precision,
        "micro_recall": micro_recall,
        "micro_f1": micro_f1,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
        "weighted_precision": weighted_precision,
        "weighted_recall": weighted_recall,
        "weighted_f1": weighted_f1,
        "hamming_loss": hamming
    }

    return metrics



def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result
    

#### Custom Class Weighting Trainer Class

In [19]:
# Reference: https://discuss.huggingface.co/t/mullti-label-text-classification/44233/3
class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights: Optional[Dict[int, float]] = None, processing_class: Optional[AutoTokenizer] = None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            # dict --> FloatTensor
            if isinstance(class_weights, dict):
                # Convert to list of values and then to FloatTensor
                class_weights = tensor(list(class_weights.values()), dtype=torch.float)
                logging.info(f"Converted class_weights to FloatTensor: {class_weights}")
            elif not isinstance(class_weights, FloatTensor):
                raise ValueError("class_weights must be a dict or a FloatTensor")
            class_weights = class_weights.to(self.args.device)

        self.loss_fct = BCEWithLogitsLoss(pos_weight=class_weights)

        # If processing_class is passed, use it
        if processing_class is not None:
            self.processing_class = processing_class  # Handling tokenizer using processing_class argument
        else:
            self.processing_class = None

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.
        Subclass and override for custom behavior.
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        try:
            loss = self.loss_fct(outputs.logits.view(-1, model.num_labels), labels.view(-1, model.num_labels))
        except AttributeError:  # DataParallel
            loss = self.loss_fct(outputs.logits.view(-1, model.module.num_labels), labels.view(-1, model.num_labels))

        return (loss, outputs) if return_outputs else loss

### Train and Validation

In [20]:
# train_subset = tokenized_dataset["train"].select(range(100))
# eval_subset = tokenized_dataset["validation"].select(range(100))

In [21]:
# class TableLoggerCallback(TrainerCallback):
#     def __init__(self):
#         self.epoch_logs = []  # Store metrics for each epoch

#     def on_epoch_end(self, args, state, control, **kwargs):
#         logs = kwargs.get("metrics", {})
#         epoch = state.epoch
        
#         # Extract all specified metrics (default to 0.0 if not found)
#         micro_precision = logs.get("eval_micro_precision", 0.0)
#         micro_recall = logs.get("eval_micro_recall", 0.0)
#         micro_f1 = logs.get("eval_micro_f1", 0.0)
#         macro_precision = logs.get("eval_macro_precision", 0.0)
#         macro_recall = logs.get("eval_macro_recall", 0.0)
#         macro_f1 = logs.get("eval_macro_f1", 0.0)
#         weighted_precision = logs.get("eval_weighted_precision", 0.0)
#         weighted_recall = logs.get("eval_weighted_recall", 0.0)
#         weighted_f1 = logs.get("eval_weighted_f1", 0.0)
#         hamming_loss = logs.get("eval_hamming_loss", 0.0)

#         # Append metrics to the list
#         self.epoch_logs.append([
#             epoch, micro_precision, micro_recall, micro_f1,
#             macro_precision, macro_recall, macro_f1,
#             weighted_precision, weighted_recall, weighted_f1, hamming_loss
#         ])

#         # Print table header
#         print("\nEpoch\tMicro_Precision\tMicro_Recall\tMicro_F1\tMacro_Precision\tMacro_Recall\tMacro_F1\tWeighted_Precision\tWeighted_Recall\tWeighted_F1\tHamming_Loss")
#         for row in self.epoch_logs:
#             print(f"{int(row[0])}\t{row[1]:.6f}\t{row[2]:.6f}\t{row[3]:.6f}\t"
#                   f"{row[4]:.6f}\t{row[5]:.6f}\t{row[6]:.6f}\t"
#                   f"{row[7]:.6f}\t{row[8]:.6f}\t{row[9]:.6f}\t{row[10]:.6f}")

# table_logger_callback = TableLoggerCallback()

In [24]:
trainer = WeightedTrainer(
    model,
    training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    # callbacks=[table_logger_callback]
)

In [25]:
trainer.train()

  0%|          | 0/33354 [00:00<?, ?it/s]

c:\Users\skpar\Desktop\Learning\ml\.env\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.2730763554573059,
 'eval_micro_precision': 0.0,
 'eval_micro_recall': 0.0,
 'eval_micro_f1': 0.0,
 'eval_macro_precision': 0.0,
 'eval_macro_recall': 0.0,
 'eval_macro_f1': 0.0,
 'eval_weighted_precision': 0.0,
 'eval_weighted_recall': 0.0,
 'eval_weighted_f1': 0.0,
 'eval_hamming_loss': 0.04428571428571428,
 'eval_runtime': 0.288,
 'eval_samples_per_second': 347.165,
 'eval_steps_per_second': 45.131,
 'epoch': 3.0}

#### Evaluate on test

In [50]:
# test_subset = tokenized_dataset["test"].select(range(100))

In [ ]:
logits, preds, metrics = trainer.predict(tokenized_dataset['test'])
preds

  0%|          | 0/13 [00:00<?, ?it/s]

array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [91]:
report = classification_report(tokenized_dataset['test'], preds, target_names=labels, zero_division=0)
print("Classification Report:\n", report)

Classification Report:
                 precision    recall  f1-score   support

    admiration       1.00      1.00      1.00         8
     amusement       1.00      1.00      1.00         7
         anger       1.00      1.00      1.00         1
     annoyance       1.00      1.00      1.00         6
      approval       1.00      1.00      1.00         3
        caring       1.00      1.00      1.00         3
     confusion       1.00      1.00      1.00         3
     curiosity       1.00      1.00      1.00         4
        desire       1.00      1.00      1.00         2
disappointment       1.00      1.00      1.00         1
   disapproval       1.00      1.00      1.00         8
       disgust       0.00      0.00      0.00         0
 embarrassment       0.00      0.00      0.00         0
    excitement       1.00      1.00      1.00         2
          fear       1.00      1.00      1.00         6
     gratitude       1.00      1.00      1.00         9
         grief       0.